In [ ]:
# Standard Library Imports
import os
import tarfile
import json
import time

# Third-Party Library Imports
import numpy as np
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from tqdm import tqdm  # For progress bar
import wandb
from PIL import ImageOps

# PyTorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split

# PyTorch-related Libraries
from torchvision.transforms import Resize, Pad
import torchvision.transforms.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau

from torchvision.transforms import InterpolationMode

In [ ]:
wandb.init(
    project="Deep-Learning-Exam",
    name="base-model",
    dir="./wandb_logs",
    config={
        "version": "v1.0.0",
        "learning_rate": 1e-4,
        "architecture": "UNet",
        "encoder": "resnet34",
        "dataset": "Kvasir-Instrument",
    },
    settings=wandb.Settings(init_timeout=900)
)

In [ ]:
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")

# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
    print(f"CUDA Version: {torch.version.cuda}")
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
else:
    print("Using CPU")

print(f"Using device: {device}")


In [ ]:
# Define dataset paths
dataset_dir = '../data/kvasir-instrument'
images_tar = os.path.join(dataset_dir, 'images.tar.gz')
masks_tar = os.path.join(dataset_dir, 'masks.tar.gz')
train_txt = os.path.join(dataset_dir, 'train.txt')
test_txt = os.path.join(dataset_dir, 'test.txt')
images_dir = os.path.join(dataset_dir, 'images')
masks_dir = os.path.join(dataset_dir, 'masks')
bbox_json = os.path.join(dataset_dir, 'bboxes.json')

# Function to extract tar.gz files
def extract_tar(tar_path, extract_path):
    if not os.path.exists(extract_path):
        with tarfile.open(tar_path, "r:gz") as tar:
            tar.extractall(path=extract_path)
            print(f"Extracted {tar_path} to {extract_path}.")
    else:
        print(f"Directory {extract_path} already exists. Skipping extraction.")

# Function to load file splits
def load_split(file_path):
    return [line.strip() for line in open(file_path, 'r')] if os.path.exists(file_path) else []

# Verify files based on splits
def verify_files(filenames, directory, extensions=('.jpg', '.jpeg', '.png')):
    return [os.path.join(directory, f + ext) for f in filenames for ext in extensions if os.path.exists(os.path.join(directory, f + ext))]

# Extract dataset
extract_tar(images_tar, images_dir)
extract_tar(masks_tar, masks_dir)

# Load train and test splits
train_filenames = load_split(train_txt)
test_filenames = load_split(test_txt)

# Load bboxes.json
with open(bbox_json, 'r') as f:
    bboxes = json.load(f)



In [ ]:
class KvasirInstrumentDataset(Dataset):
    def __init__(self, images_dir, masks_dir, image_ids, target_size=(576, 576)):
        self.images_dir = images_dir
        self.masks_dir = masks_dir
        self.image_ids = image_ids
        self.target_size = target_size
        self.resize_to_target = Resize(self.target_size)

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        image_path = os.path.join(self.images_dir, f"{image_id}.jpg")
        mask_path = os.path.join(self.masks_dir, f"{image_id}.png")

        # Load and process the image
        image = Image.open(image_path).convert('RGB')
        mask = Image.open(mask_path).convert('L')

        # Pad to square to prevent stretching of the images.
        image, mask = self.pad_to_square(image, mask)
        
        # Initialize separate resize transforms for images and masks
        self.resize_to_target_image = Resize(self.target_size, interpolation=InterpolationMode.BILINEAR)
        self.resize_to_target_mask = Resize(self.target_size, interpolation=InterpolationMode.NEAREST)

        # Resize to target size
        image = self.resize_to_target_image(image)
        mask = self.resize_to_target_mask(mask)


        # Convert to tensors
        image = torch.tensor(np.array(image, dtype=np.float32) / 255.0).permute(2, 0, 1)  # [C, H, W]
        mask = torch.tensor(np.array(mask, dtype=np.float32) / 255.0).unsqueeze(0)  # [1, H, W]

        return image, mask

    def pad_to_square(self, image, mask):
        w, h = image.size
        max_side = max(w, h)
        left = (max_side - w) // 2
        top = (max_side - h) // 2
        right = max_side - w - left
        bottom = max_side - h - top
        padding = (left, top, right, bottom)

        image = ImageOps.expand(image, border=padding, fill=0)
        mask = ImageOps.expand(mask, border=padding, fill=0)

        return image, mask


In [ ]:
torch.manual_seed(42)

# Initialize dataset and dataloader
train_size = int(0.8 * len(train_filenames))
val_size = len(train_filenames) - train_size
train_ids, val_ids = random_split(train_filenames, [train_size, val_size])

# Create datasets
train_dataset = KvasirInstrumentDataset(images_dir, masks_dir, train_ids)
val_dataset = KvasirInstrumentDataset(images_dir, masks_dir, val_ids)
test_dataset = KvasirInstrumentDataset(images_dir, masks_dir, test_filenames)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

In [ ]:
class UNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=1):
        super(UNet, self).__init__()
        
        # Encoder
        self.enc1 = self.conv_block(in_channels, 64)
        self.enc2 = self.conv_block(64, 128)
        self.enc3 = self.conv_block(128, 256)
        self.enc4 = self.conv_block(256, 512)
        
        # Bottleneck
        self.bottleneck = self.conv_block(512, 1024, dropout=0.1)
        
        # Decoder
        self.upconv4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.dec4 = self.conv_block(1024, 512, dropout=0.2)
        
        self.upconv3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.dec3 = self.conv_block(512, 256, dropout=0.1)
        
        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec2 = self.conv_block(256, 128)
        
        self.upconv1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec1 = self.conv_block(128, 64)
        
        # Final Output
        self.final = nn.Conv2d(64, out_channels, kernel_size=1)
        
    def conv_block(self, in_channels, out_channels, dropout=None):
        layers = [
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
        ]
        if dropout is not None:
            layers.append(nn.Dropout2d(p=dropout))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        # Encoder
        enc1 = self.enc1(x)
        enc2 = self.enc2(self.pool(enc1))
        enc3 = self.enc3(self.pool(enc2))
        enc4 = self.enc4(self.pool(enc3))
        
        # Bottleneck
        bottleneck = self.bottleneck(self.pool(enc4))
        
        # Decoder
        dec4 = self.upconv4(bottleneck)
        dec4 = torch.cat((dec4, enc4), dim=1)
        dec4 = self.dec4(dec4)
        
        dec3 = self.upconv3(dec4)
        dec3 = torch.cat((dec3, enc3), dim=1)
        dec3 = self.dec3(dec3)
        
        dec2 = self.upconv2(dec3)
        dec2 = torch.cat((dec2, enc2), dim=1)
        dec2 = self.dec2(dec2)
        
        dec1 = self.upconv1(dec2)
        dec1 = torch.cat((dec1, enc1), dim=1)
        dec1 = self.dec1(dec1)
        
        return self.final(dec1)

    def pool(self, x):
        return nn.MaxPool2d(kernel_size=2, stride=2)(x)


In [ ]:
model = UNet(in_channels=3, out_channels=1).to(device)

# Define the loss function and optimizer
criterion = nn.BCEWithLogitsLoss()  # For binary segmentation tasks
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)

In [ ]:
# Evaluation metrics
def iou_score(preds, masks):  # Intersection Over Union
    intersection = (preds * masks).sum((1, 2, 3))
    union = ((preds + masks) > 0).sum((1, 2, 3))
    return (intersection / union).mean().item()

def dice_coefficient(preds, masks):
    intersection = (preds * masks).sum((1, 2, 3))
    dice = (2 * intersection) / (preds.sum((1, 2, 3)) + masks.sum((1, 2, 3)))
    return dice.mean().item()

def precision(preds, masks):
    tp = (preds * masks).sum((1, 2, 3))  # True Positives
    fp = ((preds * (1 - masks))).sum((1, 2, 3))  # False Positives
    return (tp / (tp + fp + 1e-7)).mean().item()

def recall(preds, masks):
    tp = (preds * masks).sum((1, 2, 3))  # True Positives
    fn = ((1 - preds) * masks).sum((1, 2, 3))  # False Negatives
    return (tp / (tp + fn + 1e-7)).mean().item()

def f1_score(precision, recall):
    return 2 * (precision * recall) / (precision + recall + 1e-7)

def pixel_accuracy(preds, masks):
    correct = (preds == masks).sum((1, 2, 3))
    total = torch.numel(preds[0])  # Total pixels per image
    return (correct / total).mean().item()


In [ ]:
num_epochs = 25

# Define the training loop
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs, device, patience=3):
    best_val_loss = float('inf')
    epochs_no_improve = 0
    model.to(device)

    total_training_start = time.time()

    for epoch in range(num_epochs):
        epoch_start = time.time()
        epoch_loss = 0
        epoch_iou = 0
        epoch_dice = 0
        epoch_precision = 0
        epoch_recall = 0
        epoch_pixel_acc = 0

        print(f"Epoch {epoch + 1}/{num_epochs}")
        
        for images, masks in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
            images, masks = images.to(device, non_blocking=True), masks.to(device, non_blocking=True)
            
            # Forward pass
            outputs = model(images)
            preds = torch.sigmoid(outputs)
            loss = criterion(outputs, masks)
            
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
            
            # Threshold predictions at 0.5
            binary_preds = (preds > 0.5).float()

            # Calculate metrics
            epoch_iou += iou_score(binary_preds, masks)
            epoch_dice += dice_coefficient(binary_preds, masks)
            epoch_precision += precision(binary_preds, masks)
            epoch_recall += recall(binary_preds, masks)
            epoch_pixel_acc += pixel_accuracy(binary_preds, masks)
        
        # Average metrics over the epoch
        avg_loss = epoch_loss / len(train_loader)
        avg_iou = epoch_iou / len(train_loader)
        avg_dice = epoch_dice / len(train_loader)
        avg_precision = epoch_precision / len(train_loader)
        avg_recall = epoch_recall / len(train_loader)
        avg_pixel_acc = epoch_pixel_acc / len(train_loader)
        avg_f1 = f1_score(avg_precision, avg_recall)
        
        print(f"Epoch {epoch + 1}, Loss: {avg_loss:.4f}, IoU: {avg_iou:.4f}, Dice: {avg_dice:.4f}, Precision: {avg_precision:.4f}, Recall: {avg_recall:.4f}, F1: {avg_f1:.4f}, Pixel Acc: {avg_pixel_acc:.4f}")
        
        # Log to wandb
        wandb.log({
            "train_loss": avg_loss,
            "train_iou": avg_iou,
            "train_dice": avg_dice,
            "train_precision": avg_precision,
            "train_recall": avg_recall,
            "train_f1": avg_f1,
            "train_pixel_acc": avg_pixel_acc,
        })

        
        # Validation phase
        model.eval()  # Set model to evaluation mode
        val_loss = 0
        val_iou = 0
        val_dice = 0

        with torch.no_grad():
            for images, masks in val_loader:
                images, masks = images.to(device, non_blocking=True), masks.to(device, non_blocking=True)
                outputs = model(images)
                preds = torch.sigmoid(outputs)
                loss = criterion(outputs, masks)
                val_loss += loss.item()
                
                # Threshold predictions
                binary_preds = (preds > 0.8).float()
                
                # Calculate metrics
                val_iou += iou_score(binary_preds, masks)
                val_dice += dice_coefficient(binary_preds, masks)
        
        # Average validation metrics
        avg_val_loss = val_loss / len(val_loader)
        avg_val_iou = val_iou / len(val_loader)
        avg_val_dice = val_dice / len(val_loader)

        
        print(f"Validation Loss: {avg_val_loss:.4f}, IoU: {avg_val_iou:.4f}, Dice: {avg_val_dice:.4f},")
        
        # Scheduler step
        scheduler.step(avg_val_loss)

        # Early stopping
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print('Early stopping triggered')
                break
            
        model.train()  # Set model back to training mode for next epoch

    total_training_time = time.time() - total_training_start
    
    print("Training complete")

    wandb.log({"total_training_time": total_training_time})


In [ ]:
# Training
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs, device, patience=3)

In [ ]:
def eval_model(model, dataloader, device):
    model.eval()  # Set model to evaluation mode
    total_iou = 0
    total_dice = 0
    total_precision = 0
    total_recall = 0
    total_pixel_acc = 0
    results = []  # Store images, masks, and predictions for visualization

    with torch.no_grad():  # Disable gradient computation for evaluation
        for images, masks in tqdm(dataloader, desc="Evaluating"):
            images, masks = images.to(device, non_blocking=True), masks.to(device, non_blocking=True)
            outputs = model(images)
            preds = torch.sigmoid(outputs)  # Apply sigmoid to get probabilities

            # Threshold predictions at 0.5
            binary_preds = (preds > 0.5).float()

            # Store the results for visualization
            results.append((images.cpu(), masks.cpu(), binary_preds.cpu()))

            # Calculate metrics
            total_iou += iou_score(binary_preds, masks)
            total_dice += dice_coefficient(binary_preds, masks)
            total_precision += precision(binary_preds, masks)
            total_recall += recall(binary_preds, masks)
            total_pixel_acc += pixel_accuracy(binary_preds, masks)

    # Average metrics
    avg_iou = total_iou / len(dataloader)
    avg_dice = total_dice / len(dataloader)
    avg_precision = total_precision / len(dataloader)
    avg_recall = total_recall / len(dataloader)
    avg_pixel_acc = total_pixel_acc / len(dataloader)
    avg_f1 = f1_score(avg_precision, avg_recall)

    print(f"Final Model Accuracy - IoU: {avg_iou:.4f}, Dice: {avg_dice:.4f}, "
          f"Precision: {avg_precision:.4f}, Recall: {avg_recall:.4f}, "
          f"F1: {avg_f1:.4f}, Pixel Acc: {avg_pixel_acc:.4f}")

    # Log the evaluation metrics as scalars
    wandb.log({
        "eval_iou": avg_iou,
        "eval_dice": avg_dice,
        "eval_precision": avg_precision,
        "eval_recall": avg_recall,
        "eval_f1": avg_f1,
        "eval_pixel_acc": avg_pixel_acc,
    })

    return results  # Return the results for visualization


In [ ]:
def visualize_results(results, num_samples=3):
    for i in range(num_samples):
        images, masks, preds = results[i]
        for j in range(images.shape[0]):  # Iterate over batch size
            plt.figure(figsize=(12, 4))
            
            # Calculate metrics
            iou = iou_score(preds[j].unsqueeze(0), masks[j].unsqueeze(0))
            dice = dice_coefficient(preds[j].unsqueeze(0), masks[j].unsqueeze(0))
            
            plt.subplot(1, 3, 1)
            plt.imshow(images[j].permute(1, 2, 0))  # Convert [C, H, W] to [H, W, C]
            plt.title("Original Image")
            plt.axis('off')
            
            plt.subplot(1, 3, 2)
            plt.imshow(masks[j][0], cmap='gray')
            plt.title("Ground Truth Mask")
            plt.axis('off')
            
            plt.subplot(1, 3, 3)
            plt.imshow(preds[j][0], cmap='gray')
            plt.title(f"Predicted Mask\nIoU: {iou:.4f}, Dice: {dice:.4f}")
            plt.axis('off')
            
            plt.show()
            if i >= num_samples - 1:
                break


In [ ]:
# Load the best model
#model.load_state_dict(torch.load('best_model.pth'))

# Evaluate the model
results = eval_model(model, test_loader, device)

# Visualize results
visualize_results(results, num_samples=3)
wandb.finish()

In [ ]:
# Cleanup Section to Clear Memory with Memory Tracking

import gc
import psutil

# Function to get current memory usage in GB
def get_memory_usage_gb():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / (1024 ** 3)  # Convert bytes to GB

# Memory usage before cleanup
memory_before = get_memory_usage_gb()

# List of variables to delete
vars_to_delete = [
    'model',
    'train_dataset',
    'val_dataset',
    'test_dataset',
    'train_loader',
    'val_loader',
    'test_loader',
    'optimizer',
    'scheduler',
    'criterion',
    'results',
    'preds',
    'binary_preds',
    'outputs',
    'images',
    'masks',
]

for var in vars_to_delete:
    if var in globals():
        del globals()[var]

# Clear CUDA cache if using GPU
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# Collect garbage
gc.collect()

# Memory usage after cleanup
memory_after = get_memory_usage_gb()

# Display how much memory was cleared
memory_cleared = memory_before - memory_after
print(f"Memory before cleanup: {memory_before:.2f} GB")
print(f"Memory after cleanup: {memory_after:.2f} GB")
print(f"Memory cleared: {memory_cleared:.2f} GB")
